In [ ]:
# pretty much the exact same thing that y'all implemented in the homework, except instead of automatically
# printing out every single course, this has been converted into a method that will accept a specific
# link and then return the pertinent information

# something important to keep in mind is that this method is not robust *at all*. If you input any link that
# is not an Illinois course URL, it will produce some super weird output and probably break. In the final
# project, you can start off assuming user input will be valid, but if time permits, always try to make your
# code as robust as possible

def scraper(url): 

    client = ScrapingAntClient(token='231c4750fbfe4194aa1ccd2e4ea2911a')

    page_content = client.general_request(url).content

    soup = BeautifulSoup(page_content)
    tags = soup.findAll("a")
    output = []
    for item in tags:
        if "Unavailable/Padding" in str(item):
            continue
        else:
            output.append(item)
    
    completeList = []
    for item in output:
        ## eliminate the remainder unnecessary lines
        if "Available" in str(item):
        ## split the string to take the string before "- Available" because python split method only have output as a list so i had to do it this way
            value = str(item).split("- Available")
           ## split the string to take the string after '"'
            value1 = str(value[0]).split('"')
            completeList.append(value1[1])
    return completeList;    

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## method to add the location description for the rooms
def nameAdder(url, name):
    namedList = []
    unnamedList = scraper(url)
    for item in unnamedList:
        item = item + "(" + name + ")"
        namedList.append(item)
    return namedList
fullList = []
## this part is very inefficient but i don't know how else
chemLib = nameAdder('https://uiuc.libcal.com/spaces?lid=5903', 'Chemistry Library')
Funk = nameAdder('https://uiuc.libcal.com/spaces?lid=3604', 'Funk ACES Library')
grainger = nameAdder('https://uiuc.libcal.com/spaces?lid=3606', 'Grainger Engineering Library')
internationalarea = nameAdder('https://uiuc.libcal.com/spaces?lid=5766', 'International and Area Studies')
mainLib = nameAdder('https://uiuc.libcal.com/spaces?lid=3608', 'Main Library')
mediaCom = nameAdder('https://uiuc.libcal.com/spaces?lid=3474', 'Media Commons Pro Gear Rental')
musicLib = nameAdder('https://uiuc.libcal.com/spaces?lid=3153', 'Music & Performing Arts Library')
SHELib = nameAdder('https://uiuc.libcal.com/spaces?lid=3605', 'Social Science Health and Education Library')
UGL = nameAdder('https://uiuc.libcal.com/spaces?lid=3152', 'Undergraduate Library')

fullList = chemLib + Funk + grainger + internationalarea + mainLib + mediaCom + musicLib + SHELib + UGL

 

for item in fullList:
    print(item)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dictionary = {}
for item in fullList:
  timeAndPlace = item.split(" - ")
  if (timeAndPlace[0] in dictionary):
    dictionary[timeAndPlace[0]] += ", " + timeAndPlace[1].rstrip()
  else:
    dictionary.update({timeAndPlace[0]: timeAndPlace[1].rstrip()})
print(dictionary)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def finder(time, date, location):
    rawList = []
    toReturn = []
    ## configuring the input so it can be used as a key
    input = time + " " + date
    ## scrape all available locations at input time
    try: 
        ##browsing the dictionary for the appropriate values
        valueList = dictionary[input]
        ##returned values is a long string so it's needed to be split into a list
        rawList = valueList.split(", ")
    ## if nothing is found
    except:
        toReturn.append("Not found")
    ## since the user only needs certain location, the raw list is filtered
    for item in rawList:
        for x in location:
            if x in item:
                toReturn.append(item)
    return toReturn    

In [ ]:
# basic imports, just like you would normally use in a python file
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, url_for, flash, redirect
#from flask_table import Table, Col

# this sets up your flask app - you can treat this and the above imports and basic starter syntax
# that you will always use anytime you want to create a new flask app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'superdupersecretkeythatnobodywouldeverguess'

# this app.route handles the index.html page - most other routes will require a specific file name,
# but since index is the "base" of your application, it goes without an identifier
@app.route("/", methods=('GET', 'POST'))

# creating a method below to handle any requests made to the index page
def index():
    
    # checking if the method being used is POST - consult line 11 in index.html to see where
    # we specify the method as POST. we can also change this to GET if we so choose
    if request.method == 'POST':
        
        # this will get the data inputted by the user and store it in local variables
        # look to lines 16 & 21 in index.html to determine what you should adjust below - the
        # goal is to store the data within the text boxes when the "submit" button is pressed
        date = str(request.form['date'])
        time = str(request.form['time'])
        
        # this now calls @app.route("/output/<subject>/<number>"), and we assign <subject> = subject,
        # and <number> = number. For simplicity's sake, we normally keep the variable names the same,
        # but you can change them to whatever you'd like
        return redirect(url_for('output', date = date, time = time))
    
    # this is more of a dummy return since we'll never reach this point, but just in case something
    # goes wrong, we want to reroute the user back to the index page
    return render_template('oldindex.html')

# handles calls to output.html
@app.route("/output/<date>/<time>")
def output(date, time):
    return_list = ["Chemistry Library", "Funk ACES Library", "Grainger Engineering Library", "International and Area Studies", "Main Library", "Media Commons Pro Gear Rental", "Music & Performing Arts Library", "Social Science Health and Education Library", "Undergraduate Library"]
    
    # returns the scraped data as a list to output.html
    return render_template('output.html', output_list=return_list)

    """# creates the proper link from the user's given inputs
    course_url = "https://courses.illinois.edu/schedule/2022/spring/" + subject + "/" + number
    
    # At this point, we want to call our scrape method and pass it the URL
    # We want to assign the method's output to "return_list"
    return_list = scrape(course_url)
    """

# Code taken from: https://stackoverflow.com/questions/41831929/debug-flask-server-inside-jupyter-notebook

# this snippet of code is another syntactical inclusion - it's not as important to understand everything
# that's happening here, but it essentially decides where your flask app is going to run on the local device
if __name__ == '__main__':
    from werkzeug.serving import run_simple

    # the "8080" means that the flask app will run on http://localhost:8080/
    # you can change this to any other viable port if 8080 is currently in use
    # for example, you can change the line below to read "run_simple('localhost', 9000, app)", and then
    # access the flask app through http://localhost:9000/ - the port is completely changeable and
    # does not affect your code at all beyond where you access it from
    run_simple('localhost', 8880, app)

 * Running on http://localhost:8880 (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2022 05:54:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 05:54:35] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2022 05:54:39] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [29/Apr/2022 05:54:39] "GET /output/d/d HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 05:54:39] "GET /static/css/style.css HTTP/1.1" 304 -
